# Numerical validation of controllability of ansatze in Larocca et al., *Quantum* 6 824 (2022)

In [ ]:
import os
import logging
import numpy as np
import h5py
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
logging.basicConfig(level=logging.INFO)
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
jax.config.update('jax_enable_x64', True)

from fastdla.lie_closure import lie_closure
from fastdla.generators.hea import hea_generators
from fastdla.generators.spin_glass import spin_glass_generators
from fastdla.generators.heisenberg_hva import xxz_hva_generators
from fastdla.generators.tfim_hva import tfim_1d_hva_generators
from fastdla.generators.spin_chain import magnetization_eigenspace, parity_eigenspace

LOG = logging.getLogger('main')

In [ ]:
data_file = '/data/iiyama/quantum_6.h5'

In [ ]:
if data_file and not os.path.exists(data_file):
    with h5py.File(data_file, 'w'):
        pass

### Hardware efficient ansatz

$$
\mathcal{G}_{\mathrm{HEA}} = \left\{ X_n, Y_n \right\}_{n=0}^{N_q-1} \cup
                                     \left\{ \sum_{n=0}^{N_q - 2} Z_n Z_{n+1} \right\}
$$

In [ ]:
run_calc = True
if data_file:
    with h5py.File(data_file, 'r') as source:
        if 'hea' in source:
            run_calc = False
            group = source['hea']
            nqs = group['num_qubits'][()]
            dla_dims = group['dla_dims'][()]

if run_calc:
    min_nq = 2
    max_nq = 4
    nqs = np.arange(min_nq, max_nq + 1)
    dla_dims = []

    for num_qubits in nqs:
        LOG.info('num_qubits=%d', num_qubits)
        generators = hea_generators(num_qubits=num_qubits).to_matrices()
        dla = lie_closure(generators)
        dla_dims.append(dla.shape[0])

    if data_file:
        with h5py.File(data_file, 'a') as out:
            if 'hea' not in out:
                group = out.create_group('hea')
                group.create_dataset('num_qubits', data=nqs)
                group.create_dataset('dla_dims', data=dla_dims)

In [ ]:
x = np.linspace(nqs[0], nqs[-1], 100)
plt.plot(x, (2 ** x) ** 2 - 1, label='$(2^{N_q})^2-1$')
plt.scatter(nqs, dla_dims, label='DLA dimension')
plt.yscale('log')
plt.title('Hardware-efficient ansatz')
plt.xlabel('$N_q$', fontsize=16)
plt.xticks(nqs, labels=[f'{i}' for i in nqs])
plt.legend(fontsize=14);

### Spin glass



In [ ]:
run_calc = True
if data_file:
    with h5py.File(data_file, 'r') as source:
        if 'spin_glass' in source:
            run_calc = False
            group = source['spin_glass']
            nqs = group['num_qubits'][()]
            dla_dims = group['dla_dims'][()]

if run_calc:
    min_nq = 3
    max_nq = 6
    nqs = np.arange(min_nq, max_nq + 1)
    dla_dims = []

    for num_qubits in nqs:
        print(f'num_qubits={num_qubits}')
        generators = spin_glass_generators(num_qubits=num_qubits).to_matrices()
        dla = lie_closure(generators, keep_original=False)
        dla_dims.append(dla.shape[0])

    if data_file:
        with h5py.File(data_file, 'a') as out:
            if 'spin_glass' not in out:
                group = out.create_group('spin_glass')
                group.create_dataset('num_qubits', data=nqs)
                group.create_dataset('dla_dims', data=dla_dims)

In [ ]:
x = np.linspace(nqs[0], nqs[-1], 100)
plt.plot(x, (2 ** x) ** 2 - 1, label='$(2^{N_q})^2-1$')
plt.scatter(nqs, dla_dims, label='DLA dimension')
plt.yscale('log')
plt.title('Spin glass ansatz')
plt.xlabel('$N_q$', fontsize=16)
plt.xticks(nqs, labels=[f'{i}' for i in nqs])
plt.legend(fontsize=14);

### $XXZ$ model (subspace controllable)

In [ ]:
run_calc = True
if data_file:
    with h5py.File(data_file, 'r') as source:
        if 'xxz' in source:
            run_calc = False
            group = source['xxz']
            nspins = group['num_spins'][()]
            subspace_dims = group['subspace_dims'][()]
            dla_dims = group['dla_dims'][()]

if run_calc:
    min_nspin = 4
    max_nspin = 10
    nspins = np.arange(min_nspin, max_nspin + 1, 2)
    parity = 1
    subspace_dims = np.zeros((nspins.shape[0], max_nspin // 2 + 1), dtype=int)
    dla_dims = np.zeros((nspins.shape[0], max_nspin // 2 + 1), dtype=int)

    for ispin, num_qubits in enumerate(nspins):
        print(f'num_spins={num_qubits}')
        generators = xxz_hva_generators(num_qubits).to_matrices()
        parity_subspace = parity_eigenspace(parity, num_spins=num_qubits, npmod=jnp)
        for imag, magnetization in enumerate(range(0, num_qubits + 1, 2)):
            print(f'magnetization={magnetization}')
            subspace = magnetization_eigenspace(magnetization, basis=parity_subspace, npmod=jnp)
            generators_reduced = jnp.einsum('ij,gik,kl->gjl', subspace.conjugate(), generators,
                                            subspace)
            dla = lie_closure(generators_reduced, keep_original=False)
            subspace_dims[ispin, imag] = subspace.shape[1]
            dla_dims[ispin, imag] = dla.shape[0]

    if data_file:
        with h5py.File(data_file, 'a') as out:
            if 'xxz' not in out:
                group = out.create_group('xxz')
                group.create_dataset('num_spins', data=nspins)
                group.create_dataset('subspace_dims', data=subspace_dims)
                group.create_dataset('dla_dims', data=dla_dims)

In [ ]:
for imag in range(nspins[-1] // 2 + 1):
    magnetization = imag * 2
    mask = nspins >= magnetization
    plt.plot(nspins[mask], subspace_dims[mask][:, imag], label='Invariant subspace dimension')
    plt.scatter(nspins[mask], dla_dims[mask][:, imag], label='DLA dimension')
plt.yscale('log')
plt.title('XXZ model HVA')
plt.xlabel('$N_q$', fontsize=16)
plt.xticks(nspins, labels=[f'{i}' for i in nspins])
plt.legend(fontsize=14);

### $XXZ$ model (subspace uncontrollable)

In [ ]:
run_calc = True
if data_file:
    with h5py.File(data_file, 'r') as source:
        if 'xxz_u' in source:
            run_calc = False
            group = source['xxz_u']
            nspins = group['num_spins'][()]
            subspace_dims = group['subspace_dims'][()]
            dla_dims = group['dla_dims'][()]

if run_calc:
    min_nspin = 4
    max_nspin = 10
    nspins = np.arange(min_nspin, max_nspin + 1, 2)
    parity = 1
    magnetization = 0
    subspace_dims = np.zeros(nspins.shape[0], dtype=int)
    dla_dims = np.zeros(nspins.shape[0], dtype=int)

    for ispin, num_qubits in enumerate(nspins):
        print(f'num_spins={num_qubits}')
        generators = xxz_hva_generators(num_qubits, subspace_controllable=False).to_matrices()
        subspace = parity_eigenspace(parity, num_spins=num_qubits, npmod=jnp)
        subspace = magnetization_eigenspace(magnetization, basis=subspace, npmod=jnp)
        generators_reduced = jnp.einsum('ij,gik,kl->gjl', subspace.conjugate(), generators,
                                        subspace)
        dla = lie_closure(generators_reduced, keep_original=False)
        subspace_dims[ispin] = subspace.shape[1]
        dla_dims[ispin] = dla.shape[0]

    if data_file:
        with h5py.File(data_file, 'a') as out:
            if 'xxz_u' not in out:
                group = out.create_group('xxz_u')
                group.create_dataset('num_spins', data=nspins)
                group.create_dataset('subspace_dims', data=subspace_dims)
                group.create_dataset('dla_dims', data=dla_dims)

In [ ]:
plt.plot(nspins, subspace_dims, label='Invariant subspace dimension')
plt.scatter(nspins, dla_dims, label='DLA dimension')
plt.yscale('log')
plt.title('$XXZ_U$ model HVA')
plt.xlabel('$N_q$', fontsize=16)
plt.xticks(nspins, labels=[f'{i}' for i in nspins])
plt.legend(fontsize=14);

### TFIM HVA

In [ ]:
run_calc = True
if data_file:
    with h5py.File(data_file, 'r') as source:
        if 'tfim' in source:
            run_calc = False
            group = source['tfim']
            nspins = group['num_spins'][()]
            subspace_dims = group['subspace_dims'][()]
            dla_dims = group['dla_dims'][()]

if run_calc:
    min_nspin = 4
    max_nspin = 10
    nspins = np.arange(min_nspin, max_nspin + 1, 2)
    parity = 1
    magnetization = 0
    subspace_dims = np.zeros(nspins.shape[0], dtype=int)
    dla_dims = np.zeros(nspins.shape[0], dtype=int)

    for ispin, num_qubits in enumerate(nspins):
        print(f'num_spins={num_qubits}')
        generators = xxz_hva_generators(num_qubits, subspace_controllable=False).to_matrices()
        subspace = parity_eigenspace(parity, num_spins=num_qubits, npmod=jnp)
        subspace = magnetization_eigenspace(magnetization, basis=subspace, npmod=jnp)
        generators_reduced = jnp.einsum('ij,gik,kl->gjl', subspace.conjugate(), generators,
                                        subspace)
        dla = lie_closure(generators_reduced, keep_original=False)
        subspace_dims[ispin] = subspace.shape[1]
        dla_dims[ispin] = dla.shape[0]

    if data_file:
        with h5py.File(data_file, 'a') as out:
            if 'xxz_u' not in out:
                group = out.create_group('xxz_u')
                group.create_dataset('num_spins', data=nspins)
                group.create_dataset('subspace_dims', data=subspace_dims)
                group.create_dataset('dla_dims', data=dla_dims)